<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/missions_fits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FITS**

![sciences](https://raw.githubusercontent.com/deltorobarba/repo/master/sciences_2000.png)

In [ ]:
!pip install astropy astroquery requests reproject -q
from astropy import units as u
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from astropy import coordinates
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
from astropy import coordinates as coords
from reproject import reproject_interp
import matplotlib.pyplot as plt

In [ ]:
# Some tasks like multi-wavelength astronomy require high memory to load data
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Runtime: {:.1f} gigabytes of available RAM (>20 GB ideal)\n'.format(ram_gb))

Runtime: 54.8 gigabytes of available RAM (>20 GB ideal)



#### **Define astronomical object and get skycoordinates**

In [ ]:
# Define object
object_name = 'M51'

*e.g. M31 (Andromeda galaxy), M104 (Sombrero galaxy), M51 (Whirlpool galaxy), M42 (Orion nebula), NGC 3372 (Carina nebula)*

In [ ]:
# Get and set skycoordinates of object
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord

skyobject = Simbad.query_object(object_name)
ra_hms = skyobject['RA'][0]
dec_dms = skyobject['DEC'][0]
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
print(skyobject)
print("\nCoordinates:")
print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

MAIN_ID      RA          DEC      RA_PREC ... COO_WAVELENGTH     COO_BIBCODE     SCRIPT_NUMBER_ID
          "h:m:s"      "d:m:s"            ...                                                    
------- ------------ ------------ ------- ... -------------- ------------------- ----------------
  M  51 13 29 52.698 +47 11 42.93       7 ...              I 2006AJ....131.1163S                1

Coordinates:
RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Convert from Right Ascension (RA) and Declination (DEC) to decimal degree
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
ra_deg = coord.ra.deg # round(coord.ra.deg, 2)
dec_deg = coord.dec.deg # round(coord.dec.deg, 2)

print("RA in degrees:", ra_deg)
print("DEC in degrees:", dec_deg)

RA in degrees: 202.46957499999996
DEC in degrees: 47.19525833333333


In [ ]:
# Convert from decimal degree to Right Ascension (RA) and Declination (DEC)
coord = SkyCoord(ra=ra_deg, dec=dec_deg, unit='deg', frame='icrs')

# RA: hours, minutes, seconds. DEC: degrees, arcminutes, arcseconds
ra_hms = coord.ra.to_string(unit='hour', sep=' ', precision=3, pad=True)
dec_dms = coord.dec.to_string(unit='deg', sep=' ', precision=2, alwayssign=True, pad=True)

print("RA (h:m:s):", ra_hms)
print("DEC (d:m:s):", dec_dms)

RA (h:m:s): 13 29 52.698
DEC (d:m:s): +47 11 42.93


In [ ]:
# Reverse check to see to which object the coordinates belong
coord = SkyCoord(ra=ra_hms, dec=dec_dms, unit=('hourangle', 'deg'), frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M  51


frame='icrs' refers to [International Celestial Ref. System](https://aa.usno.navy.mil/faq/ICRS_doc)
*  Origin is barycenter of Solar System. This means ICRS origin is not fixed to Earth but moves with Solar System's center of mass.
* Axes are defined by the positions of extragalactic sources, primarily quasars observed using Very Long Baseline Interferometry (VLBI). The primary plane is as close as possible to the mean equator at J2000.0 (January 1, 2000, 12:00 TT).  The origin of right ascension is close to the dynamical equinox of J2000.0.
* Accuracy: The ICRF3 defines the ICRS axes with an accuracy of about 20 microarcseconds.
* Realization: The International Celestial Reference Frame (ICRF) is a catalog of precise positions of extragalactic sources that provides a practical realization of the ICRS. The latest version, ICRF3, includes the positions of over 4000 extragalactic sources, mostly quasars. The ICRF is maintained by the International Earth Rotation and Reference Systems Service (IERS).
* Relationship to other systems: ICRS is closely aligned with the J2000 equatorial coordinate system, but it's more accurate and stable. Transformations exist to convert coordinates between ICRS and other celestial reference systems

In [ ]:
# Set coordinates manually and verify correct object
coord = SkyCoord('189.9976d -11.623054d', frame='icrs')
result = Simbad.query_region(coord, radius='0d0m5s')  # Search within a small radius (5 arcseconds)
if result:
    print(result['MAIN_ID'][0])  # Show main name of the object
else:
    print("No object found in SIMBAD at these coordinates.")
object_name = 'MAIN_ID'

M 104


#### **FITS File**

In [ ]:
# Option 1: Load via IRSA (for predefined object)
from astroquery.ipac.irsa import Irsa
spitzer_images = Irsa.query_sia(pos=(coord, 1 * u.arcmin), collection='spitzer_seip').to_table()

# Open a cutout image for one of the science images
irsa_image = spitzer_images[spitzer_images['dataproduct_subtype'] == 'science'][5]
with fits.open(irsa_image['access_url'], use_fsspec=True) as hdul:
    cutout = Cutout2D(hdul[0].section, position=coord, size=2 * u.arcmin, wcs=WCS(hdul[0].header))

# Open FITS file
hdul = fits.open(irsa_image['access_url'], use_fsspec=True)

# Print file information
hdul.info()

Filename: <class 'fsspec.implementations.http.HTTPFile'>
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      77   (2480, 3095)   float32   


In [ ]:
# Option 2: Load file directly
!wget https://raw.githubusercontent.com/deltorobarba/astrophysics/master/file-u-001737-5-0039.fits
fits_file = './file-u-001737-5-0039.fits'

# Open the FITS file
hdul = fits.open(fits_file)

# Print file information
hdul.info()

--2024-12-21 19:43:55--  https://raw.githubusercontent.com/deltorobarba/astrophysics/master/file-u-001737-5-0039.fits
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12447360 (12M) [application/octet-stream]
Saving to: ‘file-u-001737-5-0039.fits.4’

file-u-001737-5-003 100%[===================>]  11.87M  --.-KB/s    in 0.03s   

2024-12-21 19:43:55 (348 MB/s) - ‘file-u-001737-5-0039.fits.4’ saved [12447360/12447360]

Filename: ./file-u-001737-5-0039.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      85   (2048, 1489)   float32   
  1                1 ImageHDU         6   (2048,)   float32   
  2                1 BinTableHDU     27   1R x 3C   [49152E, 2048E, 1489E]   
  3                1 BinTableHDU     7

In [ ]:
# Access more details from primary HDU (Header/Data Unit) to get data
primary_hdu = hdul[0]
data = primary_hdu.data
print(primary_hdu.header) # Display the header to learn more about the file
print(repr(hdul[0].header))  # Science data HDU header

SIMPLE  =                    T /                                                BITPIX  =                  -32 / 32 bit floating point                          NAXIS   =                    2                                                  NAXIS1  =                 2048                                                  NAXIS2  =                 1489                                                  EXTEND  =                    T /Extensions may be present                       BZERO   =              0.00000 /Set by MRD_SCALE                                BSCALE  =              1.00000 /Set by MRD_SCALE                                TAI     =        4476739701.74 / 1st row - Number of seconds since Nov 17 1858  RA      =            313.13132 / 1st row - Right ascension of telescope boresighDEC     =           -6.213656  / 1st row - Declination of telescope boresight (dSPA     =              97.851  / 1st row - Camera col position angle wrt north (IPA     =             111.891  / 1st row

In [ ]:
# Close the FITS file
hdul.close()